## Gaussian Discriminant Analysis

with same covariance $\Sigma_1 = \Sigma_2 = \Sigma_3 ... = \Sigma_3$

In [1]:
import pandas as pd
import numpy as np

# To reset the printoptions
# np.set_printoptions(edgeitems=3,infstr='inf',linewidth=75, nanstr='nan', precision=8,suppress=False, threshold=1000, formatter=None)

np.set_printoptions(precision=4, suppress=True)

#np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### For multi-class label y ={0, 1, 2, ...}

In [2]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target
#y=(y>0).astype(int) 

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

In [5]:
class GaussianDiscAnalysis:
    """
    Fits a Linear Discriminant Analysis Model for multi-class data
    
    lamd : Preconditioner; for matrix inversion; default:1e-10
    """
    def __init__(self, lambd=1e-10):
        self.lambd = lambd
        
    def compute_phi(self, y):
        n = len(y)
        phi = dict()
        for idx in range(len(self.classes)):
            phi[idx] = (1/n) * np.sum(y==idx)
        return phi
    
    def compute_mu(self, X, y):
        mu_dict = dict()
        for idx in range(len(self.classes)):
            # Add mu for each class
            mu_dict[idx] = np.sum(X[y==idx], axis=0)/ np.sum(y==idx)
        return mu_dict

    def compute_sigma(self, X, y):
        n = len(X)
        #y = y.reshape(-1,1)
        Xmu = X.copy()
        for idx in range(len(self.classes)):
            Xmu = Xmu \
              - self.mu[idx]*np.ones_like(Xmu)*(y==idx).reshape(-1,1)
        return (1/n) * Xmu.T@Xmu
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        d = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma)
        det_sigma = np.linalg.det(self.sigma)
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(d/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        d = X.shape[1]
        self.classes = np.unique(y)
        self.mu = self.compute_mu(X, y)
        self.sigma = self.compute_sigma(X, y) + self.lambd*np.eye(d,d)
        self.phi = self.compute_phi(y)
        
    def predict_proba(self, X):
        """Computes the probability of example belonging to that class"""
        n = len(X)
        Pyi = np.zeros((n, len(self.classes)))
        
        for idx in range(len(self.classes)):
            #print(self.compute_Pxyi(X, idx))
            py_i = self.compute_Pxyi(X, idx) * self.phi[idx]
            Pyi[:, idx] = py_i
        return Pyi
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def generate_data(self, class_id, num_samples=1):
        """Generates new unseen dataset from a normal distribution
            given the mean of class and covariance
        """
        mean = self.mu[class_id]
        cov = self.sigma
        return np.random.multivariate_normal(mean, cov, num_samples)

In [6]:
GDA = GaussianDiscAnalysis()

In [7]:
GDA.fit(X_train, y_train)

In [8]:
predictions = GDA.predict(X_test)

In [9]:
predictions

array([2, 0, 1, 2, 2, 1, 2, 1, 1, 0, 1, 2, 1, 2, 1, 0, 0, 2, 2, 2, 1, 2,
       2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 1, 2, 1])

In [10]:
GDA.predict_proba(X_test)[0:5]

array([[0.    , 0.002 , 0.5969],
       [0.9563, 0.    , 0.    ],
       [0.    , 0.2022, 0.0007],
       [0.    , 0.0001, 0.002 ],
       [0.    , 0.0001, 0.0899]])

In [11]:
np.sum(predictions == y_test) / len(y_test)

0.9736842105263158

### Generate new data

In [12]:
new_data = GDA.generate_data(class_id=1, num_samples=20)

In [13]:
new_data

array([[5.7147, 2.4357, 4.1227, 1.2079],
       [7.2455, 3.1877, 5.3909, 1.4968],
       [5.31  , 3.0933, 3.5943, 1.6828],
       [6.358 , 3.4513, 4.3994, 1.8366],
       [5.1571, 2.5409, 3.9726, 1.3853],
       [5.7358, 2.4984, 3.9294, 1.2545],
       [5.1284, 3.0295, 3.2574, 1.1319],
       [5.5758, 2.6132, 3.4698, 1.1091],
       [6.3158, 2.7922, 4.1846, 1.233 ],
       [6.3129, 3.1742, 4.344 , 1.3997],
       [6.1584, 3.4345, 4.0058, 1.3491],
       [4.9919, 2.9266, 3.5858, 1.0061],
       [5.5618, 2.7038, 4.0887, 1.1935],
       [5.7704, 2.525 , 4.4426, 1.5316],
       [5.806 , 3.4761, 4.1545, 1.1918],
       [6.8181, 3.0156, 5.2776, 1.7214],
       [5.956 , 2.7093, 4.8157, 1.6467],
       [6.1889, 2.5118, 4.0617, 1.2717],
       [6.04  , 2.8614, 3.9787, 1.3567],
       [5.0691, 2.4697, 3.2304, 1.3159]])

In [14]:
GDA.predict(new_data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1])